# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

In [1]:
%pip install -q anywidget ipywidgets wiki3_ai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%gui asyncio

In [3]:
# Import the library
from wiki3_ai import LanguageModel

In [4]:
LanguageModel.widget()

In [5]:
import wiki3_ai
wiki3_ai.__version__

'0.5.1'

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [6]:
# Check if the API is available
await LanguageModel.availability()

'available'

## Get Model Parameters

Let's see what parameters the model supports.

In [7]:
params = await LanguageModel.params()

if params:
    print(f"Default temperature: {params.get("defaultTemperature")}")
    print(f"Max temperature: {params.get("maxTemperature")}")
    print(f"Default top-K: {params.get("defaultTopK")}")
    print(f"Max top-K: {params.get("maxTopK")}")
else:
    print("Model parameters not available")

params

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


{'defaultTopK': 3,
 'maxTopK': 128,
 'defaultTemperature': 1,
 'maxTemperature': 2}

## Create LanguageModel Session

In [8]:
lm = await LanguageModel.create()
lm

## Simple Prompt

Create a session and send a simple prompt.

In [9]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Clean, clear syntax flows,
Logic blooms, a coded grace,
Worlds built line by line. 



## System Prompt

Use a system prompt to set the context for the conversation.

In [10]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

Replace `'your_file.csv'` with the actual filename. This opens the file, creates a CSV reader, and iterates through each row, printing it.  `csv.DictReader` is also useful for accessing data by column name.






In [11]:
result = await assistant_session.prompt("Please return just the code to read a CSV file in Python.  Include detailed comments.")
print(result)

```python
import csv

def read_csv(filename):
  """Reads a CSV file and returns its contents.

  Args:
    filename: The name of the CSV file to read.

  Returns:
    A list of lists, where each inner list represents a row in the CSV file.
    Returns an empty list if the file is empty or an error occurs.
  """
  try:
    with open(filename, 'r', newline='') as file:  # Open the file in read mode ('r') with newline='' to handle different line endings
      reader = csv.reader(file)  # Create a CSV reader object
      data = list(reader)  # Convert the reader object to a list of lists
      return data
  except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
    return []
  except Exception as e:
    print(f"An error occurred: {e}")
    return []

# Example usage (replace 'your_file.csv' with your filename)
# data = read_csv('your_file.csv')
# if data:
#   for row in data:
#     print(row)
```



## Streaming Response

Stream the response as it's generated.

In [12]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: Why did the computer get glasses? 

Because it needed to improve its website! 



In [13]:
async for chunk in assistant_session.prompt_streaming("Read me a long poem please."):
    print(chunk, end="", flush=True)
print()

Okay, here's a longer poem for you. It's a bit whimsical and explores themes of memory, dreams, and the passage of time.  I've aimed for something evocative.



**The Clockwork Heart of Yesterday**



The attic dust, a silver haze,
Catches the sun in bygone days.
A trunk sits closed, with latch of brass,
Holding echoes of what was.
A faded photograph, a whispered name,
A flicker of a half-forgotten flame.

The clockwork heart of yesterday,
Beats softly, though the gears decay.
Each tick a memory, a gentle chime,
Of laughter lost to the flow of time.
A child’s forgotten wooden toy,
A silent witness to a vanished joy.

The scent of lavender, a ghostly trace,
Of hands that smoothed a beloved face.
A letter penned with careful hand,
Across a distant, promised land.
Words etched in ink, a tender plea,
A yearning for what used to be.

Dreams like butterflies, on fragile wing,
Flutter through the chambers memory brings.
They dance with shadows, light and shade,
A tapestry of moments, gently l

## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [14]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

Assistant: A list comprehension is a concise way to create lists in Python. It's essentially a shorthand for creating a new list based on an existing iterable (like another list, tuple, or range).  It combines the loop and `append` steps into a single line.

Here's the basic syntax:

```python
new_list = [expression for item in iterable if condition]
```

* **expression:** What you want to put *in* the new list. It can be a simple transformation of `item`.
* **item:** Represents each element in the iterable.
* **iterable:** The sequence you're iterating over (e.g., a list, range, string).
* **condition (optional):**  A filter that determines which items from the `iterable` will be included in the `new_list`.



**Example:**

```python
numbers = [1, 2, 3, 4, 5]

# Create a new list containing the squares of the numbers
squares = [x**2 for x in numbers]
print(squares)  # Output: [1, 4, 9, 16, 25]


# Create a new list containing only the even numbers
even_numbers = [x for x in numbers if

In [15]:
# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

Assistant: ```python
# Example 1:  Squaring numbers

numbers = [1, 2, 3, 4, 5]
squared_numbers = [x**2 for x in numbers]  # Square each number
print(squared_numbers)  # Output: [1, 4, 9, 16, 25]

# Example 2: Extracting vowels from a string

text = "Hello, World!"
vowels = [char for char in text if char.lower() in 'aeiou'] # Get vowels, case-insensitive
print(vowels)  # Output: ['e', 'o', 'o']

# Example 3:  Converting strings to uppercase

words = ["apple", "banana", "cherry"]
uppercase_words = [word.upper() for word in words]
print(uppercase_words) # Output: ['APPLE', 'BANANA', 'CHERRY']

# Example 4: Conditional transformation
numbers = [1, 2, 3, 4, 5, 6]
even_odd = ["Even" if x % 2 == 0 else "Odd" for x in numbers]
print(even_odd) # Output: ['Odd', 'Even', 'Odd', 'Even', 'Odd', 'Even']
```

These examples demonstrate how list comprehensions can be used to perform different operations on lists in a concise manner.  They're particularly useful for simple transformations and filtering

## Check Token Usage

Monitor how many tokens you've used.

In [16]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 1510/9216 tokens

This prompt would use approximately 13 tokens


## Structured Output

Use JSON schema to get structured responses.

In [17]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [18]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

print("=== Branch 1 ===")
async for chunk in branch1.prompt_streaming("The dragon was friendly and helpful."):
    print(chunk, end="", flush=True)
print("=== End of 1 ===")

print("=== Branch 2 ===")
async for chunk in branch2.prompt_streaming("The dragon was fierce and terrifying."):
    print(chunk, end="", flush=True)
print("=== End of 2 ===")

=== Branch 1 ===
Once upon a time, nestled amongst the jagged peaks of the Whisperwind Mountains, lived a dragon named Zephyr. But Zephyr wasn’t like the dragons of legend. He wasn't hoarding gold or terrifying villages. Zephyr hoarded… melodies. 

He wasn't a fearsome beast, scales of obsidian and eyes of fire. Instead, Zephyr shimmered with scales the color of a twilight sky – deep purples, bruised indigos, and streaks of shimmering rose gold. His eyes weren’t fiery; they held the gentle glow of a distant nebula. And instead of roaring, Zephyr *hummed*. 

He was incredibly friendly, always ready with a comforting hum or a helpful nudge. He’d often use his powerful wings to clear paths through snowy drifts for travelers, or warm chilly mountain passes with gentle currents of air. If someone lost their way, Zephyr would listen to their worried murmurs and hum a soothing melody, subtly guiding them back to the right path. He was a benevolent guardian, a gentle giant with a kind heart.



## Cleanup

Destroy sessions when you're done to free up resources.

In [19]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
